In [16]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression # 이진 분류
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sklearn
import Titanic_preprocessing

In [31]:
# def get_category(age):
#     cat = ''
#     if age <= -1: cat = 'Unknown'
#     elif age <= 5: cat = 'Baby'
#     elif age <= 12: cat = 'Child'
#     elif age <= 18: cat = 'Teenager'
#     elif age <= 25: cat = 'Student'
#     elif age <= 35: cat = 'Young Adult'
#     elif age <= 60: cat = 'Adult'
#     else: cat = 'Elderly'
#     return cat

# def fillna(df):
#     df['Age'].fillna(df['Age'].mean(), inplace=True)
#     df['Cabin'].fillna('N', inplace=True)
#     df['Embarked'].fillna('N', inplace=True)
#     df['Fare'].fillna(0, inplace=True)
#     return df

# def drop_features(df):
#     df.drop(['PassengerId', 'Ticket', 'Name'], axis=1, inplace=True)
#     return df

# def format_features(df):
#     df['Cabin'] = df['Cabin'].str[:1]
#     features = ['Sex', 'Cabin', 'Embarked']
#     for f in features:
#         encoder = LabelEncoder()
#         df[f] = encoder.fit_transform(df[f])
#     return df

# def preprocessing(df):
#     df = fillna(df)
#     df = drop_features(df)
#     df = format_features(df)
#     return df


In [13]:
titanic_df = pd.read_csv('./kaggle/titanic/train.csv')


# titanic_df.groupby(['Sex', 'Survived'])['Survived'].count()

# sns.barplot(x='Sex', y='Survived', data=titanic_df)
# sns.barplot(x='Pclass', y='Survived', data=titanic_df, hue='Sex')

# titanic_df['Age_cat'] = titanic_df['Age'].apply(get_category)
# titanic_df['Age_cat'] = titanic_df['Age'].apply(lambda x: get_category(x))
# titanic_df['Age_cat'] = list(map(lambda x: get_category(x), titanic_df['Age']))

# group_name = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Elderly']
# sns.barplot(x='Age_cat', y='Survived', hue='Sex', data=titanic_df, order=group_name) # order: xticks, 순서

# titanic_df['Sex'].value_counts(), titanic_df['Cabin'].value_counts(), titanic_df['Embarked'].value_counts()

titanic_df = Titanic_preprocessing.preprocessing(titanic_df)

X = titanic_df[titanic_df.columns.difference(['Survived'])]  # drop을 써서 Survived 만 뺴도 될듯
Y = titanic_df[['Survived']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=11)

In [32]:
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear') # penalty: 과적합 완화, # fit_intercept: 바이어스,절편을 찾는가, solver: 알고리즘 종류

dt_clf.fit(X_train, Y_train.values.reshape(-1, ))
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier acc: {:.4f}'.format(accuracy_score(Y_test, dt_pred)))

rf_clf.fit(X_train, Y_train.values.reshape(-1, ))
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifier acc: {:.4f}'.format(accuracy_score(Y_test, rf_pred)))

lr_clf.fit(X_train, Y_train.values.reshape(-1, ))
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression acc: {:.4f}'.format(accuracy_score(Y_test, lr_pred)))


DecisionTreeClassifier acc: 0.7877
RandomForestClassifier acc: 0.8547
LogisticRegression acc: 0.8659


In [38]:
def exec_kfold(clf, df_X, df_Y, fold=5):
    kfold = KFold(n_splits=fold)
    scores = []
    for i, (train_index, test_index) in enumerate(kfold.split(df_X)):
        X_train, X_test = df_X.values[train_index], df_X.values[test_index]
        Y_train, Y_test = df_Y.values[train_index], df_Y.values[test_index]

        clf.fit(X_train, Y_train.reshape(-1, ))
        pred = clf.predict(X_test)
        scores.append(accuracy_score(Y_test, pred))
        print('교차검증 {} acc: {}'.format(i, scores[-1]))
    mean_score = np.mean(scores)
    print('평균 acc: {:.4f}'.format(mean_score))

In [48]:
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')


for i in [dt_clf, rf_clf, lr_clf]:
    print(i)
    exec_kfold(i, X, Y, fold=5)
    print(cross_val_score(i, X, Y.values.reshape(-1, ), cv=5, scoring='accuracy'))
    print(np.mean(cross_val_score(i, X, Y.values.reshape(-1, ), cv=5, scoring='accuracy')))
    print()
    

DecisionTreeClassifier(random_state=11)
교차검증 0 acc: 0.7541899441340782
교차검증 1 acc: 0.7640449438202247
교차검증 2 acc: 0.7865168539325843
교차검증 3 acc: 0.7696629213483146
교차검증 4 acc: 0.8202247191011236
평균 acc: 0.7789
[0.74301676 0.7752809  0.79775281 0.78651685 0.84269663]
0.7890527901575545

RandomForestClassifier(random_state=11)
교차검증 0 acc: 0.7653631284916201
교차검증 1 acc: 0.7921348314606742
교차검증 2 acc: 0.8258426966292135
교차검증 3 acc: 0.7808988764044944
교차검증 4 acc: 0.8595505617977528
평균 acc: 0.8048
[0.78212291 0.79213483 0.84831461 0.7752809  0.85393258]
0.8103571652752495

LogisticRegression(solver='liblinear')
교차검증 0 acc: 0.7932960893854749
교차검증 1 acc: 0.7921348314606742
교차검증 2 acc: 0.7752808988764045
교차검증 3 acc: 0.7471910112359551
교차검증 4 acc: 0.8426966292134831
평균 acc: 0.7901
[0.7877095  0.79213483 0.7752809  0.76404494 0.82022472]
0.7878789780930262



In [52]:
param = {
    'max_depth': [2, 3, 5, 10], 
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 5, 8]
}
grid_clf = GridSearchCV(dt_clf, param, scoring='accuracy', cv=5)
grid_clf.fit(X_train, Y_train)

print('GridSearchCV 최적 하이퍼 파라미터: {}'.format(grid_clf.best_params_))
print('GridSearchCV 최고 acc: {}'.format(grid_clf.best_score_))
best_clf = grid_clf.best_estimator_

best_pred = best_clf.predict(X_test)
print('acc: {:.4f}'.format(accuracy_score(Y_test, best_pred)))

GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 acc: 0.7991825076332119
acc: 0.8715
